In [1]:
import pandas as pd
import numpy as np

In [2]:
import os

In [3]:
os.getcwd()

'C:\\Users\\Mohd Noman Ansari'

In [4]:
os.chdir("C:\\Users\\Mohd Noman Ansari\\Downloads\\fake-news")

In [5]:
os.getcwd()

'C:\\Users\\Mohd Noman Ansari\\Downloads\\fake-news'

In [6]:
df=pd.read_csv('train.csv')

In [7]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [8]:
df.dropna(inplace = True)

In [9]:
df.drop('id',axis = 1 ,inplace = True)

In [16]:
df['text'][0]

'House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It By Darrell Lucus on October 30, 2016 Subscribe Jason Chaffetz on the stump in American Fork, Utah ( image courtesy Michael Jolley, available under a Creative Commons-BY license) \nWith apologies to Keith Olbermann, there is no doubt who the Worst Person in The World is this week–FBI Director James Comey. But according to a House Democratic aide, it looks like we also know who the second-worst person is as well. It turns out that when Comey sent his now-infamous letter announcing that the FBI was looking into emails that may be related to Hillary Clinton’s email server, the ranking Democrats on the relevant committees didn’t hear about it from Comey. They found out via a tweet from one of the Republican committee chairmen. \nAs we now know, Comey notified the Republican chairmen and Democratic ranking members of the House Intelligence, Judiciary, and Oversight committees that his agency was reviewing emai

In [11]:
X = df.drop('label',axis=1)

In [12]:
y = df['label']

In [13]:
X

,title,author,text
0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...
20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [14]:
y

0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 18285, dtype: int64

In [17]:
X.shape

(18285, 3)

In [18]:
y.shape

(18285,)

In [19]:
import tensorflow as tf

In [20]:
tf.__version__

'2.12.0'

In [21]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [22]:
voc_size = 5000

In [23]:
messages = X.copy()

In [24]:
messages.reset_index(inplace=True)

In [25]:
import nltk
import re

In [26]:
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\Mohd Noman
[nltk_data]     Ansari\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [27]:
from nltk.stem import WordNetLemmatizer
lm = WordNetLemmatizer()

In [28]:
corpus=[]
for i in range(0,len(messages)):
    #print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    review = [lm.lemmatize(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [30]:
corpus[0]

'house dem aide even see comey letter jason chaffetz tweeted'

In [32]:
from tensorflow.keras.preprocessing.text import one_hot

In [33]:
vocab_size = 10000

In [34]:
ohr = [one_hot(words,voc_size) for words in corpus]

54

In [39]:
sent_len=20
embedded_docs = pad_sequences(ohr , padding = 'pre', maxlen = sent_len)

In [40]:
print(embedded_docs)

[[   0    0    0 ... 1274 2125 2378]
 [   0    0    0 ... 1354 1682 2794]
 [   0    0    0 ... 4807 3549 1990]
 ...
 [   0    0    0 ... 2512 3249   94]
 [   0    0    0 ...  522  671 1262]
 [   0    0    0 ...  943 3104 1375]]


In [41]:
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length = sent_len))
model.add(LSTM(100))
model.add(Dense(1,activation = 'sigmoid'))
model.compile(loss = 'binary_crossentropy' , optimizer = 'adam' , metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [42]:
X_final = np.array(embedded_docs)
y_final = np.array(y)

In [43]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X_final,y_final,test_size=0.33,random_state=42)

In [44]:
model.fit(X_train,y_train,validation_data=(X_test,y_test), batch_size=64,epochs=10)

Epoch 1/10
192/192 [==============================] - 6s 20ms/step - loss: 0.3392 - accuracy: 0.8327 - val_loss: 0.1985 - val_accuracy: 0.9163
Epoch 2/10
192/192 [==============================] - 3s 18ms/step - loss: 0.1359 - accuracy: 0.9498 - val_loss: 0.1956 - val_accuracy: 0.9191
Epoch 3/10
192/192 [==============================] - 3s 18ms/step - loss: 0.0890 - accuracy: 0.9682 - val_loss: 0.2351 - val_accuracy: 0.9171
Epoch 4/10
192/192 [==============================] - 3s 17ms/step - loss: 0.0566 - accuracy: 0.9810 - val_loss: 0.2619 - val_accuracy: 0.9128
Epoch 5/10
192/192 [==============================] - 3s 17ms/step - loss: 0.0321 - accuracy: 0.9907 - val_loss: 0.3100 - val_accuracy: 0.9147
Epoch 6/10
192/192 [==============================] - 3s 17ms/step - loss: 0.0167 - accuracy: 0.9959 - val_loss: 0.3709 - val_accuracy: 0.9165
Epoch 7/10
192/192 [==============================] - 3s 18ms/step - loss: 0.0101 - accuracy: 0.9970 - val_loss: 0.4638 - val_accuracy: 0.9152

In [ ]:
from tensorflow.keras.layers import Dropout

embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length = sent_len))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation = 'sigmoid'))
model.compile(loss = 'binary_crossentropy' , optimizer = 'adam' , metrics = ['accuracy'])
print(model.summary())

In [52]:
pred = np.where(model.predict(X_test) > 0.5 ,1,0)

189/189 [==============================] - 1s 3ms/step


In [53]:
pred

array([[1],
       [0],
       [0],
       ...,
       [0],
       [1],
       [1]])

In [54]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test,pred))

[[3189  230]
 [ 363 2253]]


In [55]:
from sklearn.metrics import classification_report
print(classification_report(y_test , pred))

              precision    recall  f1-score   support

           0       0.90      0.93      0.91      3419
           1       0.91      0.86      0.88      2616

    accuracy                           0.90      6035
   macro avg       0.90      0.90      0.90      6035
weighted avg       0.90      0.90      0.90      6035

